### Kroing-Pennt model
###### $ -\dfrac{\hbar^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} x^2} + V(x)\psi = H\psi = E\psi $ ( $ Ax = \lambda x $ )

In [1]:
###############################################################################
# Imports
###############################################################################
import numpy as np
import numpy.linalg as lin
import math
import ipywidgets as widgets
from ipywidgets import interact_manual, IntSlider, FloatSlider, Layout
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import physical constants from scipy
from scipy.constants import physical_constants

# Set numpy print options for debugging (optional)
np.set_printoptions(threshold=784, linewidth=np.inf)

###############################################################################
# Global Constants and Unit Conversions
###############################################################################
# 1 Bohr = 0.52917721067 Angstrom, so 1 Angstrom = 1/0.52917721067 Bohr ~ 1.88973
bohr_radius_ang = physical_constants['Bohr radius'][0] * 1e10  # in Angstrom
ang2bohr = 1 / bohr_radius_ang  # ~1.88973

# Hartree energy in eV
har2ev = physical_constants['Hartree energy'][0] / physical_constants['electron volt'][0]  # ~27.21140795

# Global variable for Laplacian finite difference accuracy
laplacianDim = 3

###############################################################################
# Laplacian Functions (Finite Difference Method)
###############################################################################
def laplacianCoeff(laplacianDim):
    """
    Compute the coefficients for the Laplacian operator using the Finite Difference Method (FDM).

    This function constructs the finite difference coefficients for the second derivative 
    using central difference formulas. The method is based on solving a linear system 
    that relates the coefficients to the derivatives of a function at grid points.
    """
    size = 2 * laplacianDim + 1
    a = np.zeros((size, size))
    c = np.zeros(size)
    c[2] = math.factorial(2)  # For second derivative, 2! = 2
    for i in range(size):
        for j in range(size):
            a[i, j] = (j - laplacianDim) ** i
    inva = lin.inv(a)
    b = inva @ c  # b = a^-1 * c
    lcoeff = b[laplacianDim:size]
    return lcoeff

def laplacian(ngridx, laplacianDim, dx):
    """
    Construct the Laplacian operator matrix for a 1D grid using the Finite Difference Method.
    Uses periodic boundary conditions.
    """
    lcoeff = laplacianCoeff(laplacianDim)
    loprt = np.zeros((ngridx, ngridx))
    for i in range(ngridx):
        for j in range(-laplacianDim, laplacianDim + 1):
            k = i + j
            if k >= ngridx:
                k -= ngridx
            elif k < 0:
                k += ngridx
            loprt[i, k] = lcoeff[abs(j)] / (dx ** 2)
    return loprt

###############################################################################
# Potential and Hamiltonian Functions
###############################################################################
def potential(ngridx, num_well, pot_shape=1, pot_height_eV=25, barrier_width=2, well_width=2):
    """
    Set up the potential for the Kronig-Penney model.
    
    In each cell (period), the barrier region has width 'barrier_width' and 
    the well region has width 'well_width'. The potential barrier height is pot_height_eV (in eV), 
    and the well region is 0 eV.
    
    Parameters:
    -----------
    ngridx : int
        Number of grid points.
    num_well : int
        Number of cells (periods) in the structure.
    pot_shape : int
        Potential shape selector (1 corresponds to the Kronig-Penney model).
    pot_height_eV : float
        Barrier height in eV.
    barrier_width : float
        Barrier width (in Angstrom).
    well_width : float
        Well width (in Angstrom).
    
    Returns:
    --------
    pot_grid : ndarray
        The potential at each grid point (in Hartree).
    """
    pot_height_har = pot_height_eV / har2ev
    ngridx_per_cell = ngridx // num_well
    pot_grid = np.zeros(ngridx)
    if pot_shape == 1:
        # 전체 cell을 barrier potential으로 채운 후,
        # 각 cell의 well 영역(왼쪽 부분, 비율 = well_width/(barrier_width+well_width))은 0으로 설정
        pot_grid[:] = pot_height_har
        fraction_well = well_width / (barrier_width + well_width)
        for i in range(num_well):
            start = int(i * ngridx_per_cell)
            end = int(i * ngridx_per_cell + fraction_well * ngridx_per_cell)
            pot_grid[start:end] = 0
    return pot_grid

def hamiltonian(ngridx, laplacianDim, dx, num_well, pot_height_eV=25, pot_shape=1, 
                barrier_width=2, well_width=2):
    """
    Construct the Hamiltonian matrix H = -1/2 * Laplacian + Potential.
    """
    pot_op = np.zeros((ngridx, ngridx))
    pot_1d = potential(ngridx, num_well, pot_shape, pot_height_eV, barrier_width, well_width)
    laplacian_op = laplacian(ngridx, laplacianDim, dx)
    np.fill_diagonal(pot_op, pot_1d)
    ham_op = -laplacian_op / 2. + pot_op
    return ham_op

def time_inde_schr_eq(ngridx, pot_shape, pot_height_eV, barrier_width, well_width, dx, num_well):
    """
    Solve the time-independent Schrödinger equation and return eigenvalues and eigenvectors.
    """
    ham_op = hamiltonian(ngridx, laplacianDim, dx, num_well, pot_height_eV, pot_shape, barrier_width, well_width)
    eigval, eigvec = np.linalg.eigh(ham_op)
    return eigval, eigvec

###############################################################################
# Plot Functions Using Plotly (Modified)
###############################################################################
def plot_Kroing_Penny_supercell(eigval, eigvec, ngridx, boxL, pot_shape, 
                                pot_height_eV, barrier_width, well_width, num_bands, num_well):
    """
    Plot eigenstates and potential using Plotly.
    """
    # x coordinate in Angstrom
    x = np.linspace(-boxL/2, boxL/2, ngridx) / ang2bohr  # Convert from Bohr to Angstrom

    # Determine indices for eigenstates (min and max for each band)
    eig_list = []
    list_color = ['red', 'green', 'blue', 'magenta', 'yellow', 'cyan']
    list_line = ['solid', 'dash']

    for i in range(num_bands):
        eig_list.append(i * num_well + np.argmin(eigval[i * num_well : (i + 1) * num_well]))
        eig_list.append(i * num_well + np.argmax(eigval[i * num_well : (i + 1) * num_well]))

    # Create subplots
    fig = make_subplots(rows=1, cols=2, column_widths=[0.75, 0.25],
                        subplot_titles=("Eigenstates & Potential", ""))

    # Left subplot: Eigenstates and potential
    for i in eig_list:
        eigstate = (eigvec[:, i] - eigvec[0, i]) * 10 + np.ones(ngridx) * eigval[i] * har2ev
        fig.add_trace(go.Scatter(x=x, y=eigstate,
                                 mode='lines',
                                 line=dict(color=list_color[(i // num_well) % len(list_color)],
                                           dash=list_line[(i + 1) % num_well - 1]),
                                 name=f"Eig {i}"),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=[x[0], x[-1]],
                                 y=[eigval[i] * har2ev, eigval[i] * har2ev],
                                 mode='lines',
                                 line=dict(color='gray', dash='dot'),
                                 showlegend=False),
                      row=1, col=1)

    # Plot potential
    pot = potential(ngridx, num_well, pot_shape, pot_height_eV, barrier_width, well_width)
    fig.add_trace(go.Scatter(x=x, y=pot * har2ev,
                             mode='lines',
                             line=dict(color='black'),
                             name="Potential"),
                  row=1, col=1)

    fig.update_xaxes(title_text="Box [Angstrom]", row=1, col=1)
    y_max = eigval[eig_list[-1]] * 1.1 * har2ev
    fig.update_yaxes(title_text="Energy [eV]", range=[0, y_max], row=1, col=1)

    # Right subplot: Band contour using rectangles and centered annotations
    for i in range(num_bands):
        y_min = eigval[i * num_well] * har2ev
        y_max_band = eigval[(i + 1) * num_well - 1] * har2ev
        fig.add_shape(type="rect",
                      xref="x2 domain", yref="y2",
                      x0=0, x1=1, y0=y_min, y1=y_max_band,
                      fillcolor=list_color[i % len(list_color)],
                      opacity=0.5, line_width=0,
                      row=1, col=2)
        fig.add_annotation(x=0.4, y=(y_min + y_max_band) / 2,
                           xref="x2", yref="y2",
                           text=f"Min: {y_min:.2f} eV<br>Max: {y_max_band:.2f} eV",
                           showarrow=False,
                           font=dict(size=10, color="black"),
                           xanchor="left",
                           row=1, col=2)

    fig.update_xaxes(visible=False, row=1, col=2)
    fig.update_yaxes(title_text="Energy [eV]", range=[0, y_max], row=1, col=2)

    fig.update_layout(title_text="Kronig-Penney: Eigenstates, Potential", width=1000, height=600)
    fig.show()

def Kroing_Penny_analytic(eigval, pot_height_eV, barrier_width, well_width, num_bands, boxL, num_well):
    """
    Solve the analytic equation of the Kronig-Penney model and plot the resulting E–k diagram using Plotly.
    Here, we set a = well width and b = barrier width (converted to Bohr).
    """
    list_color = ['red', 'green', 'blue', 'magenta', 'yellow', 'cyan']
    energy = np.linspace(1e-3, eigval[(num_bands+4)*num_well - 1], 3000, endpoint=True)
    k_val = np.sqrt(2 * energy + 0j)
    q_val = np.sqrt(2 * (pot_height_eV/har2ev - energy) + 0j)
    
    # In analytic solution, let a = well width and b = barrier width (convert Angstrom -> Bohr)
    a_val = well_width * ang2bohr   # well width in Bohr
    b_val = barrier_width * ang2bohr  # barrier width in Bohr
    
    functional = ((q_val**2 - k_val**2)/(2*q_val*k_val)) * np.sinh(q_val*b_val) * np.sin(k_val*a_val) \
                 + np.cosh(q_val*b_val) * np.cos(k_val*a_val)
    
    e_low, e_high = [], []
    for i in range(len(functional) - 1):
        if (functional[i] < 1 and functional[i+1] > 1) or (functional[i] > 1 and functional[i+1] < 1):
            e_low.append(np.real(k_val[i]**2 / 2))
        if (functional[i] < -1 and functional[i+1] > -1) or (functional[i] > -1 and functional[i+1] < -1):
            e_high.append(np.real(k_val[i]**2 / 2))
    e_low.sort(); e_high.sort()
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Analytic Functional", "E–k Diagram"))
    
    fig.add_trace(go.Scatter(
        x=np.real(k_val * a_val / np.pi),
        y=np.real(functional),
        mode='lines',
        name="Functional",
        line=dict(color='black')
    ), row=1, col=1)
    fig.add_hline(y=1, line=dict(color='gray', dash='dot'), row=1, col=1)
    fig.add_hline(y=-1, line=dict(color='gray', dash='dot'), row=1, col=1)
    fig.add_shape(type="rect", xref="x1", yref="y1",
                  x0=np.min(np.real(k_val * a_val / np.pi)),
                  x1=np.max(np.real(k_val * a_val / np.pi)),
                  y0=1, y1=2, fillcolor="gray", opacity=0.2, layer="below",
                  row=1, col=1)
    fig.add_shape(type="rect", xref="x1", yref="y1",
                  x0=np.min(np.real(k_val * a_val / np.pi)),
                  x1=np.max(np.real(k_val * a_val / np.pi)),
                  y0=-2, y1=-1, fillcolor="gray", opacity=0.2, layer="below",
                  row=1, col=1)
    fig.update_xaxes(title_text="Ka/π", row=1, col=1)
    fig.update_yaxes(title_text="Functional", range=[-2, 2], row=1, col=1)
    
    kl = np.linspace(0, np.pi, 100)
    for i in range(num_bands):
        if i < len(e_high) and i < len(e_low):
            weight = np.real((e_high[i] - e_low[i]) / 2)
            shift = np.real((e_high[i] + e_low[i]) / 2)
            cos_func = (weight * (-np.cos(kl)) + shift) * har2ev
            fig.add_trace(go.Scatter(
                x=kl,
                y=np.real(cos_func),
                mode='lines+markers',
                name=f'Band {i+1}',
                marker=dict(color=list_color[i % len(list_color)])
            ), row=1, col=2)
            center_y = (np.real(e_low[i]) + np.real(e_high[i])) / 2 * har2ev
            fig.add_annotation(
                x=0.8 * np.pi, y=center_y,
                xref="x2", yref="y2",
                text=f"Min: {np.real(e_low[i]*har2ev):.2f} eV<br>Max: {np.real(e_high[i]*har2ev):.2f} eV",
                showarrow=False,
                font=dict(size=10, color=list_color[i % len(list_color)]),
                xanchor="left"
            )
    fig.update_xaxes(title_text="k(a+b)", row=1, col=2)
    fig.update_yaxes(title_text="Energy [eV]", row=1, col=2)
    
    fig.update_layout(title_text="Kronig-Penney Analytic Solutions (Bloch wave)", width=1000, height=600, showlegend=True)
    fig.show()

###############################################################################
# Interactive Widget
###############################################################################
# Define base grid resolution; ngridx will scale with the number of wells.
base_ngridx = 501  # Base number of grid points per cell

slider_layout = Layout(width='400px')
label_style = {'description_width': '150px'}

@interact_manual(
    num_bands=IntSlider(min=1, max=8, step=1, value=4, 
                        description='# of Bands:', style=label_style, layout=slider_layout),
    
    num_well=IntSlider(min=2, max=16, step=1, value=8, 
                        description='# of Cells:',  # 여기서 wells 대신 cells로 표현
                        disabled=True,
                        layout=widgets.Layout(display='none', width='400px')),
    
    pot_height_eV=FloatSlider(min=0.5, max=20, step=0.1, value=3, 
                        description='Barrier Height [eV]:', 
                        style=label_style, layout=slider_layout),
    
    barrier_width=FloatSlider(min=1, max=10, step=0.1, value=3, 
                        description='Barrier Width [Ang]:', 
                        style=label_style, layout=slider_layout),
    
    well_width=FloatSlider(min=1, max=10, step=0.1, value=3, 
                        description='Well Width [Ang]:', 
                        style=label_style, layout=slider_layout)
)
def interactive(num_bands, num_well, pot_height_eV, barrier_width, well_width):
    # Total box length (Angstrom) = number of cells * (barrier_width + well_width)
    boxL_ang = num_well * (barrier_width + well_width)
    boxL = boxL_ang * ang2bohr
    ngridx = base_ngridx * num_well
    dx = boxL / ngridx

    print(readme)

    # Solve the time-independent Schrödinger equation
    print('Solving Schrödinger equation')
    eigval, eigvec = time_inde_schr_eq(ngridx, 1, pot_height_eV, barrier_width, well_width, dx, num_well)
    
    # Plot eigenstates & potential
    print('Draw eigenvectors')
    plot_Kroing_Penny_supercell(eigval, eigvec, ngridx, boxL, 1, pot_height_eV, barrier_width, well_width, num_bands, num_well)
    
    print('Draw analytic solution (PBC)')
    Kroing_Penny_analytic(eigval, pot_height_eV, barrier_width, well_width, num_bands, boxL, num_well)

if __name__ == "__main__":
    readme = r"""
V
│     V₀
│────────────────────┐   <-- V = V₀ (Barrier)
│                    │
│                    │   
│                    │        
│                    │       
│                     ───────┐   <-- 0 eV (Well)
└────────────────────────────────→ x
<------Barrier------><--Well-->
    
    Barrier Height  = V₀ 
    Barrier Width   = a
    Well Width      = b
    Period of cell  = a + b
    """
    print(readme)

    pass


interactive(children=(IntSlider(value=4, description='# of Bands:', layout=Layout(width='400px'), max=8, min=1…


V
│     V₀
│────────────────────┐   <-- V = V₀ (Barrier)
│                    │
│                    │   
│                    │        
│                    │       
│                     ───────┐   <-- 0 eV (Well)
└────────────────────────────────→ x
<------Barrier------><--Well-->
    
    Barrier Height  = V₀ 
    Barrier Width   = a
    Well Width      = b
    Period of cell  = a + b
    


## Edison - Kroing-Penney model
#### Minsu Jeong, KAIST Electrical Engineering

###### Last updated : 2024. 08. 07

### 1. Kroing-Penny model

In quantum mechanics, the particle in a one-dimensional lattice is a problem that occurs in the model of a periodic crystal lattice. 
 The potential is caused by ions in the periodic structure of the crystal creating an electromagnetic field so electrons are subject to a regular potential inside the lattice. 
 It is a generalization of the free electron model, which assumes zero potential inside the lattice.

When talking about solid materials, the discussion is mainly around crystals – periodic lattices. Here we will discuss a 1D lattice of positive ions.

The Kronig–Penney model (named after Ralph Kronig and William Penney) is a simple, idealized quantum-mechanical system that consists of an infinite periodic array of rectangular potential barriers.

The potential function is approximated by a rectangular potential.

According to Bloch's theorem, the wavefunction solution of the Schrödinger equation when the potential is periodic with a period $a$, can be written as:

$\displaystyle \psi (x)=e^{ikx}u(x), $

where $u(x)$ is a periodic function which satisfies $u(x + a) = u(x)$.





$ K = \frac{\sqrt(2mE)}{\hbar} $

$ Q = \frac{\sqrt(2m(V_{0}-E))}{\hbar} $

$ [\frac{Q^{2}-K^{2}}{2QK}] \sinh Qb \sin Ka + \cosh Qb \cos Ka = \cos k(a+b) $


ref : Kittel, p169

### 2. Solving time-independent one-dimensional (1-D) Schrödinger equation 

##### the Schrödinger equation can be solved analytically or numerically on a one-dimensional lattice of finite length using the theory of periodic differential equations.

##### by solving TISE (Time-independent Schrodinger Equation) in the finite potential well, which is periodically repeated (in the bigger infinite potential well) , 
##### So we could simulate the Kroing-Penny model and forbidden bands.

##### we plot the resulting band minima and maxima, and these will be consistent with Kroing-Penny model.




